# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

#### 2. LDA算法的基本假设是什么？

#### 3. 在TextRank算法中构建图的权重是如何得到的？

#### 4. 什么是命名实体识别？ 有什么应用场景？

#### 5.NLP主要有哪几类任务 ？

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

#### 选做 1.  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

#### 选择2. ： 电影评论分类。

在这个作业中你要完成一个电影评论分类任务。

1.数据获取。（采用爬虫技术爬取相关网页上的电影评论数据，例如猫眼电影评论，豆瓣电影评论）

2.把所获得数据分解为训练集，验证集和测试集。

3.选用相应算法构建模型，并测试。

#### 选择3：文章自动续写

在这个作业中你要完成一个文章自动续写的模型。

1.数据获取。（根据你的兴趣采用爬虫技术爬去相关网站上的文本数据内容：比如故事网站，小说网站等）

2.选取模型，并训练。

In [1]:
# 安装 bert4keras
!pip install git+https://www.github.com/bojone/bert4keras.git

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Cloning https://www.github.com/bojone/bert4keras.git to c:\users\fuliu~1.des\appdata\local\temp\pip-req-build-xzad307s
  Created wheel for bert4keras: filename=bert4keras-0.7.1-cp37-none-any.whl size=39066 sha256=765abf50768c3705e6b2ddb2080f2ffbea2c81693aa4ce5ff9c5cfb84e5de6a4
  Stored in directory: C:\Users\FULIU~1.DES\AppData\Local\Temp\pip-ephem-wheel-cache-ncpgb3um\wheels\12\58\83\8ff5c864b80c860e6d9e9e0d90c04fafca05d01d21f9f6fcba
Successfully built bert4keras


  Running command git clone -q https://www.github.com/bojone/bert4keras.git 'C:\Users\FULIU~1.DES\AppData\Local\Temp\pip-req-build-xzad307s'


In [1]:
import tensorflow as tf

https://github.com/CyberZHG/keras-bert/blob/master/demo/load_model/keras_bert_load_and_extract.ipynb

In [15]:
# 使用Bert吧

# 测试代码可用性

from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
import numpy as np
import os


pretrainedPath = 'PretrainedModelData/chinese_L-12_H-768_A-12/'

config_path = os.path.join(pretrainedPath, 'bert_config.json')
checkpoint_path = os.path.join(pretrainedPath, 'bert_model.ckpt')
dict_path = os.path.join(pretrainedPath, 'vocab.txt')
# dict_path = pretrainedPath + '/vocab.txt'

In [ ]:
print(os.path.isfile(dict_path))
print(os.path.isdir(pretrainedPath))
print(os.getcwd())
print(os.path.abspath('.'))
print(os.listdir(pretrainedPath))

tokenizer = Tokenizer(dict_path) # 建立分词器
model = build_transformer_model(config_path, checkpoint_path) # 建立模型，加载权重
print('tokenizer', tokenizer)
# 编码测试
token_ids, segment_ids = tokenizer.encode(u'语言模型')
print(model.predict([np.array([token_ids]), np.array([segment_ids])]))

In [1]:
import glob, re
import numpy as np
from tqdm import tqdm
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer, load_vocab
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, open
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder

Using TensorFlow backend.


In [13]:
novels = []

print(glob.glob('Datas/*.txt'))
for txt in glob.glob('Datas/*.txt'):
    txt = open(txt, encoding='utf-8').read()
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.replace(u'整理制作，并提供下载', '')
    txt = re.sub(u'www.*?com', '', txt)
    txt = txt.replace(u'\u3000', ' ')
    sents = []
    for t in txt.split('  '):
        for s in re.findall(u'.*?。', t):
            if len(s) <= maxlen - 2:
                sents.append(s)
    novels.append(sents)


print(len(novels))

['Datas\\星辰变.txt', 'Datas\\盘龙.txt']
2


In [16]:
# 加载并精简词表，建立分词器
token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)

[unused1] ['[unused1]']
[unused2] ['[unused2]']
[unused3] ['[unused3]']
[unused4] ['[unused4]']
[unused5] ['[unused5]']
[unused6] ['[unused6]']
[unused7] ['[unused7]']
[unused8] ['[unused8]']
[unused9] ['[unused9]']
[unused10] ['[unused10]']
[unused11] ['[unused11]']
[unused12] ['[unused12]']
[unused13] ['[unused13]']
[unused14] ['[unused14]']
[unused15] ['[unused15]']
[unused16] ['[unused16]']
[unused17] ['[unused17]']
[unused18] ['[unused18]']
[unused19] ['[unused19]']
[unused20] ['[unused20]']
[unused21] ['[unused21]']
[unused22] ['[unused22]']
[unused23] ['[unused23]']
[unused24] ['[unused24]']
[unused25] ['[unused25]']
[unused26] ['[unused26]']
[unused27] ['[unused27]']
[unused28] ['[unused28]']
[unused29] ['[unused29]']
[unused30] ['[unused30]']
[unused31] ['[unused31]']
[unused32] ['[unused32]']
[unused33] ['[unused33]']
[unused34] ['[unused34]']
[unused35] ['[unused35]']
[unused36] ['[unused36]']
[unused37] ['[unused37]']
[unused38] ['[unused38]']
[unused39] ['[unused39]']
[unu

In [17]:
data = []
pbar = tqdm(desc=u'构建语料中', total=sum(len(n) for n in novels))

for novel in novels:
    s = u''
    for i in range(len(novel)):
        for j in range(len(novel) - i):
            if len(s) + len(novel[i + j]) > maxlen - 2:
                data.append(s)
                s = u''
                break
            else:
                s += novel[i + j]
        pbar.update(1)
        if i + j >= len(novel):
            break
    if s:
        data.append(s)

pbar.close()
np.random.shuffle(data)

构建语料中: 100%|██████████| 108657/108657 [00:01<00:00, 103929.73it/s]


In [18]:
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids = [], []
        for is_end, text in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                yield [batch_token_ids, batch_segment_ids], None
                batch_token_ids, batch_segment_ids = [], []

In [19]:
model = build_transformer_model(
    config_path,
    checkpoint_path,
    application='lm',
    keep_tokens=keep_tokens,  # 只保留keep_tokens中的字，精简原字表
)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    10432512    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [20]:
# 交叉熵作为loss，并mask掉输入部分的预测
y_true = model.input[0][:, 1:]  # 目标tokens
y_mask = model.get_layer('Embedding-Token').output_mask[:, 1:]  # 目标mask
y_mask = K.cast(y_mask, K.floatx())  # 转为浮点型
y_pred = model.output[:, :-1]  # 预测tokens，预测与目标错开一位
cross_entropy = K.sparse_categorical_crossentropy(y_true, y_pred)
cross_entropy = K.sum(cross_entropy * y_mask) / K.sum(y_mask)

model.add_loss(cross_entropy)
model.compile(optimizer=Adam(1e-5))

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training_utils.py:819: UserWarning: Output MLM-Proba missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to MLM-Proba.
  'be expecting any data to be passed to {0}.'.format(name))


In [21]:
class StoryCompletion(AutoRegressiveDecoder):
    """基于随机采样的故事续写
    """
    @AutoRegressiveDecoder.set_rtype('probas')
    def predict(self, inputs, output_ids, step):
        token_ids = inputs[0]
        token_ids = np.concatenate([token_ids, output_ids], 1)
        segment_ids = np.zeros_like(token_ids)
        return model.predict([token_ids, segment_ids])[:, -1]

    def generate(self, text, n=1, topk=5):
        token_ids, _ = tokenizer.encode(text)
        results = self.random_sample([token_ids[:-1]], n, topk)  # 基于随机采样
        return [text + tokenizer.decode(ids) for ids in results]


story_completion = StoryCompletion(
    start_id=None, end_id=tokenizer._token_end_id, maxlen=maxlen
)


In [25]:
def just_show():
    s1 = u'当晚两人在一家小客店中宿歇。张无忌躺在炕上，越想越是担心，走到赵敏窗外，但听她呼吸调匀，正自香梦沉酣。'
    s2 = u'虚竹飞身跃上松树的枝干，只见段延庆的钢杖深深嵌在树枝之中，全凭一股内力粘劲，挂住了下面四人，内力之深厚，实是非同小可。虚竹伸左手抓住钢杖，提将上来。'
    s3 = u'杨过居住在侠客岛，是令狐冲的弟子，武器是金蛇剑。'
    for s in [s1, s2, s3]:
        t = story_completion.generate(s)
        print(u'输入: %s' % s)
        print(u'结果: %s\n' % ('\n'.join(t)))


class Evaluate(keras.callbacks.Callback):
    def __init__(self):
        self.lowest = 1e10

    def on_epoch_end(self, epoch, logs=None):
        # 保存最优
        if logs['loss'] <= self.lowest:
            self.lowest = logs['loss']
            model.save_weights('./best_model.weights')
        # 演示效果
        just_show()

In [ ]:
maxlen = 256
batch_size = 16
steps_per_epoch = 10
epochs = 10

In [ ]:
evaluator = Evaluate()
train_generator = data_generator(data, batch_size)

model.fit_generator(
    train_generator.forfit(),
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    callbacks=[evaluator]
)

Epoch 1/10
10/10 [==============================] - 241s 24s/step - loss: 6.3499
输入: 当晚两人在一家小客店中宿歇。张无忌躺在炕上，越想越是担心，走到赵敏窗外，但听她呼吸调匀，正自香梦沉酣。
结果: 当晚两人在一家小客店中宿歇。张无忌躺在炕上，越想越是担心，走到赵敏窗外，但听她呼吸调匀，正自香梦沉酣。的的不的一，人，。一一一一一一一一一一一一一一一一，一一一一一一一一一一一一一一。一一一一一一一一一一。一一一一一一一一一一一一一一一一一一一一一的一一一一。。一一一一一一一一一一一一一一一一一一一一一一一一一一的一一一一一一一。。。一一一一一一一一一一个一一一一一一一一一一一一一一个一一一一一一一一一一一一一一一一一一一二一一一一一一。。一一一个个，一一一一一一一二一一一一一一一一一一一一一一一一一一一。一一一一一一一一一一一一一一一一一一一一，一一一

In [ ]:
just_show()

为什么bert4keras能做到模型自己实现，模型参数却从外部加载？

3.展示一些你模型的输出例子。